In [ ]:
from scipy import stats
from pprint import pprint
import numpy as np
from scipy import spatial
from operator import mul

In [ ]:
PATH = r'/home/fazza/Documents/TLN-LAB-PART-2/resources/annotations_fabio_andrea.txt'
PATH_NASARI = r'/home/fazza/Documents/TLN-LAB-PART-2/resources/mini_NASARI.tsv'
PATH_SEM = r'/home/fazza/Documents/TLN-LAB-PART-2/resources/SemEval17_IT_senses2synsets.txt' 

In [ ]:
def get_valutation_from_txt(path):
    andrea_list = []
    fabio_list = []
    mean_list = []
    with open(path, 'r') as file:
        for line in file.readlines():
            andrea_list.append(float(line.split('    ')[2]))
            fabio_list.append(float(line.split('    ')[3]))
            mean_list.append(float(line.split('    ')[4]))

    return andrea_list, fabio_list, mean_list

def get_words_from_txt(path):
    list_of_words = []
    with open(path, 'r') as file:
        for line in file.readlines():
            list_of_words.append((line.split('    ')[0],line.split('    ')[1]))
    return list_of_words

In [ ]:

words = get_words_from_txt(PATH)
'''
print(words)
#print(f"Lista di Andrea: {a}")
#print(f"Lista di Fabio: {f}")

print("\nSPEARMAN'S CORRELATION COEFFICIENT\n")
print(f"{stats.spearmanr(a, f).correlation}")

print("\nPEARSON'S CORRELATION COEFFICIENT\n")
print(f"{stats.pearsonr(a, f)}")'''

Recupero vettori di NASARI

In [ ]:
def nasari_to_dict(path):
    nasari_dict = {}
    with open(path, 'r', encoding="utf8") as file:
        for line in file.readlines():
            splitted_line = line.split('\t')
            nasari_dict[splitted_line[0].split('__')[0]] = splitted_line[1:]
            
    return nasari_dict

Recupero synset da SemEval17_IT_senses2synsets.txt

In [ ]:
def get_babel_syns(word, path):
    babel_list = []
    flag = False
    with open(path, 'r', encoding="utf8") as file: 
        for line in file.readlines():
            if word in line:
                flag = True
            elif flag and line[0] != '#':
                babel_list.append(line)
            else:
                flag = False
    return babel_list                

Calcolo similarità

In [ ]:
def cos_sim(vec1, vec2):
    return 1 - spatial.distance.cosine(vec1, vec2)

Calcolo il massimo tra i valori di similarità

In [84]:
def get_nasari_valutation(syn1, syn2, nasari_dict):
    max = 0.0
    max_elem = (None, None)
    for s1 in syn1:
        for s2 in syn2:
            if s1.strip('\n') in nasari_dict.keys() and s2.strip('\n') in nasari_dict.keys():
                test1 = list(map(float, nasari_dict[s1.strip('\n')]))
                test2 = list(map(float, nasari_dict[s2.strip('\n')]))
                sim = cos_sim(test1, test2)
                if sim > max:
                    max = sim
                    max_elem = (s1.strip('\n'), s2.strip('\n'))
    return max, max_elem
                

In [ ]:
nasari_dict = nasari_to_dict(PATH_NASARI)

Esecuzione

In [90]:
def mult(res):
    return res*4

result = []
result_elem = []
for tuple in words:
    syn_1 = get_babel_syns(tuple[0], PATH_SEM)
    syn_2 = get_babel_syns(tuple[1], PATH_SEM)
    
    sim_max, elements = get_nasari_valutation(syn_1, syn_2, nasari_dict)
    
    result.append(sim_max)
    result_elem.append(elements)

andrea, fabio, mean = get_valutation_from_txt(PATH)

print("\nSPEARMAN'S CORRELATION COEFFICIENT\n")
print(f"{stats.spearmanr(result, mean).correlation}")

print("\nPEARSON'S CORRELATION COEFFICIENT\n")
print(f"{stats.pearsonr(result, mean)}")


50
[('bn:00023796n', 'bn:01317296n'), ('bn:00069924n', 'bn:00074064n'), ('bn:00013447n', 'bn:01387235n'), ('bn:03237376n', 'bn:03476756n'), ('bn:02580408n', 'bn:00013173n'), ('bn:00010217n', 'bn:00031976n'), ('bn:00028682n', 'bn:00010152n'), ('bn:00005869n', 'bn:00051510n'), ('bn:00017031n', 'bn:00064468n'), ('bn:00021410n', 'bn:00069626n'), ('bn:00054739n', 'bn:02730666n'), ('bn:00002177n', 'bn:00045719n'), ('bn:00057430n', 'bn:00012682n'), ('bn:03520751n', 'bn:02131227n'), ('bn:00033657n', 'bn:00007752n'), ('bn:00013722n', 'bn:00044044n'), ('bn:00018734n', 'bn:00001697n'), ('bn:00014988n', 'bn:00059980n'), ('bn:14407671n', 'bn:00063293n'), ('bn:00081285n', 'bn:00068220n'), ('bn:00015923n', 'bn:00036583n'), ('bn:01363738n', 'bn:00081636n'), ('bn:00052888n', 'bn:00073964n'), ('bn:00068572n', 'bn:00006747n'), ('bn:00054021n', 'bn:00078920n'), ('bn:00081176n', 'bn:03777357n'), ('bn:00028863n', 'bn:00042379n'), ('bn:03252503n', 'bn:00060967n'), ('bn:00059285n', 'bn:00008280n'), ('bn:01361